# Phi-2
- Microsoft 가 발표한 Tiny LLM(???)
- 2.7B(27억) parameters
- https://huggingface.co/microsoft/phi-2

In [ ]:
# ### install (upgrade) huggingface transformers library.
# !pip install -U transformers

In [1]:
# 노트북에 보기 좋게 출력하기 위한 Utillity
from IPython.display import display
from IPython.display import Markdown

import textwrap
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

torch.set_default_device("cuda")

- 몇몇 모델은 License 문제로 huggingface API key를 요구 할 수도 있으나 Phi-2 는 완전 공개 (MIT license) 모델이라 필요치 않음

In [3]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

c:\users\zpdlc\appdata\local\programs\python\python311\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zpdlc\.cache\huggingface\hub\models--microsoft--phi-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


- Model card 의 예제는 greedy generation
- https://huggingface.co/docs/transformers/main/en/generation_strategies

In [4]:
cfg = GenerationConfig(
    max_new_tokens=200,
    do_sample=True,
    temperature=0.2,
    top_k=40,
    top_p=0.95,
    eos_token_id=model.config.eos_token_id # stop_ token과 비슷한 기능
)

inputs = tokenizer('''```python
def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, generation_config=cfg)
text = tokenizer.batch_decode(outputs)[0]
to_markdown(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


> ```python
> def print_prime(n):
>    """
>    Print all primes between 1 and n
>    """
>    for num in range(2, n+1):
>        for i in range(2, num):
>            if (num % i) == 0:
>                break
>        else:
>            print(num)
> 
> print_prime(20)
> ```
> 
> ## Exercises
> 
> 1. Write a Python function that takes a list of numbers and returns the sum of all the even numbers in the list.
> 
> ```python
> def sum_even(numbers):
>    """
>    Returns the sum of all even numbers in a list
>    """
>    total = 0
>    for num in numbers:
>        if num % 2 == 0:
>            total += num
>    return total
> 
> print(sum_even([1, 2, 3, 4, 5, 6]))
> ```
> 
> 2. Write a Python function that takes a list of strings and returns a new list containing only the

- Local 에서 LLM 을 구동시키고 API 를 제공하기위한 패키지들이 존재함
  - https://github.com/abetlen/llama-cpp-python
  - https://github.com/vllm-project/vllm

- 모델을 더 작고 가볍게 만들어 낮은 성능의 PC에서도 실행 할 수 있게 하는 quantization 이라는 방법이 있음
  - https://huggingface.co/blog/4bit-transformers-bitsandbytes

In [5]:
def sum_even(numbers):
   """
   Returns the sum of all even numbers in a list
   """
   total = 0
   for num in numbers:
       if num % 2 == 0:
           total += num
   return total

print(sum_even([1, 2, 3, 4, 5, 6]))

12
